In [ ]:
# Final driver program
# * Final Version * 
from bs4 import BeautifulSoup
import requests
import time
import copy
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import urllib.request
import pymysql
import datetime


def datesInput():
    userdate = input("(YYYY-MM-DD):")
    year, month, day = map(int, userdate.split('-'))
    userdate = datetime.date(year, month, day)

    return userdate


def check_for_div_class_count(html, class_name):
    soup = BeautifulSoup(html, 'html.parser')
    milspecs = soup.findAll('div', {'class': class_name})
    return len(milspecs)

def get_date(s):
    if s.find("div", {"class":"tBJ dyH iFc yTZ B9u zDA IZT swG"}) == None:
        return ("No Date Found")
    else:
#         print("Date: ", s.find("div", {"class":"tBJ dyH iFc yTZ B9u zDA IZT swG"}).text)

#       Format of date needs to be = "2019-02-02"
#         return (s.find("div", {"class":"tBJ dyH iFc yTZ B9u zDA IZT swG"}).text)
        return("0000-00-00")

def get_emojis(s):
    # If emojis are not found on a post
    if s.find("div", {"class":"tBJ dyH iFc yTZ pBj zDA IZT swG"}) == None:
        return 0
    else:
        # Check if emoji tag that was found contains valid data
        emoji_tag = s.findAll("div", {"class":"tBJ dyH iFc yTZ pBj zDA IZT swG"})[-1].text
        # Emoji number has potential to have a 'k' value at the end 
        if emoji_tag.isnumeric() or emoji_tag[:-1].isnumeric() or isfloat(emoji_tag[:-1]):
            if emoji_tag[-1] == 'k':
                return (float(emoji_tag[:-1]) * 1000)
            else:
                return (float(emoji_tag))
        else:
            return 0
    
def get_comments(s, h):
    # Check for 2 different types of pages for comments
    # New style of comments section
    if s.find("div", {"class":"tBJ dyH iFc yTZ B9u zDA IZT mWe"}) != None:
        if s.find("div", {"class":"tBJ dyH iFc yTZ B9u zDA IZT mWe"}).text[0].isnumeric():
            return(int(s.find("div", {"class":"tBJ dyH iFc yTZ B9u zDA IZT mWe"}).text.split()[0]))
        else:
            return 0
        
    # Old style of comments in pinterest
    else:
        comments = check_for_div_class_count(h, "VxL zI7 iyn Hsu")

        # Check if any comments exist on page
        if comments == None:
            return 0
        else:
            # If more than 2 comments, need to check for more comments
            if comments >= 2:
                try:
                    if s.find("div", {"class":"Hvp zI7 iyn Hsu"}).text:
                        additional_comments = s.find("div", {"class":"Hvp zI7 iyn Hsu"}).text.split()[0]
                        a_c = int(additional_comments)
                        comments += int(a_c)
                        return comments
                except:
                    pass
                try:
                    milspecs = s.findAll('div', {'class': "VxL zI7 iyn Hsu"})
                    return(len(milspecs))
                except:
                    return 0
            else:
                return comments
            
            
def get_image_url(s):
    all_imgs = s.find_all('img')
    return(all_imgs[1]['src'])


def get_pin_description(s):
    if s.find("div", {"class":"FNs hDW zI7 iyn Hsu"}) == None:
        return("No Description")
    else:
        return (s.find("div", {"class":"FNs hDW zI7 iyn Hsu"}).text)
    
# Determine if a value is a float  
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

# Scroll through webpage
def scroller():
    curr_height = browser.execute_script("return document.documentElement.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, " + str(curr_height) + ");")
        time.sleep(1)
        new_height = browser.execute_script("return document.documentElement.scrollHeight")
        if curr_height == new_height:
            break
        curr_height = new_height

def pinterest_scraper(h_list, s_date, e_date):
    # Server Settings
    chromeOptions = Options()
    chromeOptions.add_argument('--no-sandbox')
    chromeOptions.add_argument('--remote-debugging-port=44224')
    chromeOptions.add_argument('--disable-dev-shm-using')
    chromeOptions.add_argument('--headless')
    chromeOptions.add_argument('disable-setuid-sandbox')
    chromeOptions.add_argument('disable-gpu')
    
    delta = s_date - e_date
    
    # Conditional statements to determine how many pins are going to be scraped
    if abs(delta.days) <= 30:
        pin_amount = 20
    elif abs(delta.days) > 30 and abs(delta.days) < 90:
        pin_amount = 50
    elif abs(delta.days) >= 90:
        pin_amount = 100
    
    
    # Need to download chrome driver and place in python code path in order to work
    browser = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chromeOptions)
    
    # ESTABLISH CONNECTION WITH DB
    server = "ls-1ef1825172e62dcc237ee491d09a0c12aff562fe.cn5ycdfnko6g.us-east-1.rds.amazonaws.com"
    database_ = 'smcDB'
    username = 'dbmasteruser'
    password_ = 'q+o.H1sd$CRRZl&CSl>VK}-(~+t1ea&P'

    db = pymysql.connect(host=server, user=username, password=password_, database=database_, charset='utf8mb4',
                         cursorclass=pymysql.cursors.DictCursor, port=3306)
    cursor = db.cursor()

#     print("Database connection successfully established")
    
    # Log in information for pinterest account
    usernameStr = 'andres.dicochea@ucdenver.edu'
    passwordStr = 'SchoolRocks16'

    # Go to pinterest log in page
    browser.get("https://pinterest.com/login")
    sleep(5)

    # Enter fields to log in
    username_field = browser.find_element_by_id('email')
    username_field.clear()
    username_field.send_keys(usernameStr)
    password_field = browser.find_element_by_id('password')
    password_field.clear()
    password_field.send_keys(passwordStr)
    submit=browser.find_element_by_tag_name('form')
    submit.submit() # log into pinterest
    sleep(5)
    
    total_dates_found = 0
    total_emojis_found = 0
    total_comments_found = 0

    # Go each brands account's and get pins
    for handle in h_list:
        total_handle_comments = 0
        total_handle_dates = 0
        total_handle_emojis = 0
        
        browser.get("https://pinterest.com/" + handle + "/pins/")
        sleep(5)
        
        # Get page page source for brand handle
        page_source = browser.page_source
        page = BeautifulSoup(page_source, 'html.parser')

        all_pin_links = []
        previous_pin_amount = 0
        while 1:
            # get the html of the webpage
            page_source = browser.page_source
            page = BeautifulSoup(page_source, 'html.parser')

            # Store all pins that are found on current view of the webpage
            productLinks = [div.a for div in 
                page.findAll('div', attrs={'class' : 'Yl- MIw Hb7'})]

            # Convert all product links into https links
            for index,link in enumerate(productLinks):
                productLinks[index] = "https://pinterest.com" + link['href']

            # Append to overall list of pin links
            all_pin_links.extend(productLinks)

            # Remove duplicates in the list
            temp_list = []
            [temp_list.append(x) for x in all_pin_links if x not in temp_list]
            all_pin_links = copy.deepcopy(temp_list)

            # Break out of loop once enough pins have been gathered
            if len(all_pin_links) >= int(pin_amount) or previous_pin_amount == len(all_pin_links):
                break

            # Scroll through webpage to get pin list
            browser.execute_script("window.scrollBy(0,10000)")
            time.sleep(3)
            browser.execute_script("window.scrollBy(0,10000)")
            time.sleep(3)

            # Save previous amount of pins to avoid infinite loop
            previous_pin_amount = len(all_pin_links)

        # Cut off extra pins in pin list
        del all_pin_links[int(pin_amount):]

        for link in all_pin_links:
            sleep(10)
            browser.get(link)
            print (link)

            sleep(10)
            html = browser.page_source
            soup = BeautifulSoup(html)

            # Find date of post
            date = str(get_date(soup))

            # Find total emojis on page
            emojis = str(get_emojis(soup))

            # Find total amount of comments
            comments = str(get_comments(soup, html))
            
            # Get image for pin
            image_url = str(get_image_url(soup))
            
            # Get description for pin
            description = str(get_pin_description(soup))
            description = description.replace("'","")
            
            shares = "0"
            pins = "0"
            closeups = "0"
            saves = "0"

            
            sql = '''
            insert into smcDB.PINTEREST(BrandHandle,PostUrl,PostDate,PostText,Emoticons,Comments,Shares,Pins,Closeups,Saves,ImageUrl) values('%s','%s',
            '%s','%s','%s','%s','%s','%s','%s','%s','%s') 
            ''' % (handle, link, date, description, emojis, comments, shares, pins, closeups, saves, image_url)
            cursor.execute(sql)
            db.commit()

            print("Successfully committed to database")
    
    
# Get input handle from user
handle_list = []
handle_list.append(input("Input account name: "))
handle_list.append(input("Input account name: "))
handle_list.append(input("Input account name: "))
handle_list.append(input("Input account name: "))
handle_list.append(input("Input account name: "))

# userdate = datesInput()
# userdate1 = datesInput()


# Run scraper
pinterest_scraper(handle_list, start_date, end_date)